In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv("Wimbledon_featured_matches.csv")
# print(data)

In [17]:
# 独热编码列: 'serve_width', 'winner_shot_type'
for column in ['serve_width', 'winner_shot_type']:
    insert_loc = data.columns.get_loc(column)
    data = pd.concat([data.iloc[:,:insert_loc], pd.get_dummies(data.loc[:, [column]]), data.iloc[:,insert_loc+1:]], axis=1)
# Replace 'NCTL' with 0 and 'CTL' with 1 in 'serve_depth' column
data['serve_depth'] = data['serve_depth'].replace({'NCTL': 0, 'CTL': 1})
# Replace 'ND' with 0.5 and 'D' with 1 in return_depth
data['return_depth'] = data['return_depth'].replace({'ND': 0.5, 'D': 1})
# Fill missing values in return_depth with 0
data['return_depth'].fillna(0, inplace=True)
# Fill missing values in speed_mph with 0
data['speed_mph'].fillna(0, inplace=True)
# Fill missing values in serve_depth with 0
data['serve_depth'].fillna(0, inplace=True)
# Drop columns which are before 'server'
m,n = data.shape
dic_column = dict(zip(data.columns, range(n)))
data.drop(data.columns[list(range(dic_column["server"]))], axis=1, inplace=True)


In [18]:
data_columns_p1 = []
data_columns_p2 = []
for column in data.columns:
    if column.startswith('p1'):
        data_columns_p1.append(column)
    elif column.startswith('p2'):
        data_columns_p2.append(column)
    else:
        data_columns_p1.append(column)
        data_columns_p2.append(column)

data_p1 = data[data_columns_p1]
data_p2 = data[data_columns_p2]

In [25]:
# Add a column named 'momentum' with all values set to 0
data['momentum'] = np.nan
m,n = data.shape
dic_column = dict(zip(data.columns, range(n)))
new_match_start = []
for i in range(m):
    if data.iloc[i,dic_column["elapsed_time"]] == "00:00:00":
        new_match_start.append(i)
new_match_start.append(m-1)
print(new_match_start)
for i in range(len(new_match_start)-1):
    data.iloc[new_match_start[i], dic_column['momentum']] = 0
    start = new_match_start[i]
    end = new_match_start[i+1]
    for j in range(start+1, end+1):
        if data.iloc[j, dic_column['server']] == data.iloc[j-1, dic_column['point_victor']]:
            data.iloc[j, dic_column['momentum']] = 1
        else:
            data.iloc[j, dic_column['momentum']] = 0
data.iloc[new_match_start[-1], dic_column['momentum']] = 0

[0, 300, 501, 635, 972, 1218, 1550, 1782, 1972, 2185, 2503, 2673, 2948, 3238, 3423, 3621, 3788, 4013, 4287, 4409, 4695, 4910, 5105, 5436, 5707, 5896, 6179, 6372, 6589, 6748, 6950, 7283]


In [ ]:
# 熵权法
n, m = data.shape
data2 = pd.DataFrame(np.zeros((n, m)), columns=data.columns, index=data.index)
for j in data.columns:
    col_min = data[j].min()
    col_max = data[j].max()
    diff = col_max - col_min
    if diff == 0:
        data2[j] = 0.0001
    else:
        data2[j] = (data[j] - col_min) / diff
        data2.loc[data2[j] == 0, j] = 0.0001

p = data2.div(data2.sum(axis=0), axis=1)
e = - (p * np.log(p)).sum(axis=0) / np.log(n)
g = 1 - e
w = g / g.sum()
s = w.values @ p.values.T
ss = -np.sort(-s)
rank = np.argsort(-s) + 1

TypeError: unsupported operand type(s) for -: 'str' and 'str'